In [2]:
# Langkah 1: Install dan import library
!pip install pandas python-docx nltk Sastrawi --quiet

import pandas as pd
# from docx import Document # This library is not needed for CSV files
import re
import nltk
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

nltk.download('punkt')

# Langkah 2: Load file CSV
def extract_comments_from_csv(file_path):
    try:
        df = pd.read_csv(file_path, encoding='latin-1') # Added encoding
        return df
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return pd.DataFrame() # Return an empty DataFrame if file not found
    except UnicodeDecodeError:
        print(f"Error: UnicodeDecodeError with latin-1. Trying another encoding.")
        try:
            df = pd.read_csv(file_path, encoding='ISO-8859-1') # Try another encoding
            return df
        except UnicodeDecodeError:
            print(f"Error: UnicodeDecodeError with ISO-8859-1. Trying another encoding.")
            try:
                df = pd.read_csv(file_path, encoding='cp1252') # Try another encoding
                return df
            except Exception as e:
                print(f"Error reading file with cp1252 encoding: {e}")
                return pd.DataFrame()
        except Exception as e:
            print(f"Error reading file with ISO-8859-1 encoding: {e}")
            return pd.DataFrame()


# Langkah 3: Jalankan fungsi ekstraksi
file_path = '/content/Book1.csv' # Ensure this is the correct path to your CSV file
df = extract_comments_from_csv(file_path)


# Langkah 4: Tampilkan sebagian data jika DataFrame is not empty
if not df.empty:
    print(df.head())

    # Optional: Simpan sebagai CSV untuk keperluan selanjutnya
    df.to_csv('komentar_samsat.csv', index=False)
else:
    print("DataFrame is empty. Please check the file path and content.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 12.6 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Error: File not found at /content/Book1.csv
DataFrame is empty. Please check the file path and content.


In [3]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [4]:
import pandas as pd
import re
import nltk
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt # Import matplotlib for plotting later

# Re-load the original data and reapply preprocessing steps

# Langkah 2: Load file CSV (replicated from cell O3y5Z5M0DXj3)
def extract_comments_from_csv(file_path):
    try:
        df = pd.read_csv(file_path, encoding='latin-1') # Added encoding
        print(f"Successfully loaded data from {file_path}") # Add success message
        return df
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return pd.DataFrame() # Return an empty DataFrame if file not found
    except UnicodeDecodeError:
        print(f"Error: UnicodeDecodeError with latin-1. Trying another encoding.")
        try:
            df = pd.read_csv(file_path, encoding='ISO-8859-1') # Try another encoding
            print(f"Successfully loaded data from {file_path} with ISO-8859-1 encoding") # Add success message
            return df
        except UnicodeDecodeError:
            print(f"Error: UnicodeDecodeError with ISO-8859-1. Trying another encoding.")
            try:
                df = pd.read_csv(file_path, encoding='cp1252') # Try another encoding
                print(f"Successfully loaded data from {file_path} with cp1252 encoding") # Add success message
                return df
            except Exception as e:
                print(f"Error reading file with cp1252 encoding: {e}")
                return pd.DataFrame()
        except Exception as e:
            print(f"Error reading file with ISO-8859-1 encoding: {e}")
            return pd.DataFrame()

# Langkah 3: Jalankan fungsi ekstraksi
file_path = '/content/Book1.csv' # Ensure this is the correct path to your CSV file
df = extract_comments_from_csv(file_path)


# Langkah 4: Lanjutkan pra-pemrosesan jika DataFrame tidak kosong
if not df.empty:
    # Rename columns based on the first row (replicated from cell eaad2c82)
    new_columns = df.iloc[0].tolist()
    df.columns = new_columns
    df = df.drop(0).reset_index(drop=True)

    # Handle missing values and irrelevant rows (replicated from cells 61ef7bcd and c52fe95d)
    df.dropna(subset=['Isi Komentar'], inplace=True)
    other_cols = [col for col in df.columns if col != 'Isi Komentar']
    rows_to_drop = df[df[other_cols].isnull().all(axis=1)].index
    df.drop(rows_to_drop, inplace=True)

    # Text cleaning (replicated from cell 4ff00012)
    def clean_text(text):
        if pd.isna(text):
            return ""
        text = text.lower()
        text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
        text = re.sub(r'\d+', '', text)
        text = re.sub(r'[^\w\s]', '', text)
        text = text.strip()
        text = re.sub(r'\s+', ' ', text)
        return text

    df['Cleaned_Komentar'] = df['Isi Komentar'].apply(clean_text)

    # Tokenization (replicated from cell TevlVskiF2k1)
    try:
        nltk.data.find('tokenizers/punkt')
    except LookupError:
        nltk.download('punkt')

    def tokenize_text(text):
        if not text:
            return []
        return nltk.word_tokenize(text)

    df['Tokenized_Komentar'] = df['Cleaned_Komentar'].apply(tokenize_text)

    # Stop word removal (replicated from cell bxAhPpBrF7VZ)
    stop_word_factory = StopWordRemoverFactory()
    stop_word_remover = stop_word_remover_factory.create_stop_word_remover()

    def remove_stopwords(tokens):
        if not tokens:
            return []
        text = " ".join(tokens)
        filtered_text = stop_word_remover.remove(text)
        return filtered_text.split()

    df['Filtered_Komentar'] = df['Tokenized_Komentar'].apply(remove_stopwords)

    # Stemming (replicated from cell 6093c74f)
    stemmer_factory = StemmerFactory()
    stemmer = stemmer_factory.create_stemmer()

    def stem_tokens(tokens):
        if not tokens:
            return []
        text = " ".join(tokens)
        stemmed_text = stemmer.stem(text)
        return stemmed_text.split()

    df['Stemmed_Komentar'] = df['Filtered_Komentar'].apply(stem_tokens)


    # Instantiate the VADER sentiment intensity analyzer
    analyzer = SentimentIntensityAnalyzer()

    # Define a function to classify sentiment
    def classify_sentiment(text):
        if pd.isna(text) or not isinstance(text, str): # Handle potential non-string/NaN values
            return 'Neutral'  # Classify empty or non-string text as neutral
        scores = analyzer.polarity_scores(text)
        compound_score = scores['compound']
        if compound_score >= 0.05:
            return 'Positive'
        elif compound_score <= -0.05:
            return 'Negative'
        else:
            return 'Neutral'

    # Apply the sentiment classification function to the 'Cleaned_Komentar' column
    df['Sentiment'] = df['Cleaned_Komentar'].apply(classify_sentiment)

    # Calculate the value counts of the 'Sentiment' column
    sentiment_counts = df['Sentiment'].value_counts()

    # Create a bar plot for sentiment distribution
    plt.figure(figsize=(8, 6))
    sentiment_counts.plot(kind='bar', color=['green', 'red', 'blue'])
    plt.title('Sentiment Distribution of Comments')
    plt.xlabel('Sentiment')
    plt.ylabel('Number of Comments')
    plt.xticks(rotation=0) # Keep x-axis labels horizontal
    plt.tight_layout() # Adjust layout to prevent labels overlapping
    plt.show()

    # Display the head of the DataFrame with the relevant columns
    display(df[['Cleaned_Komentar', 'Sentiment']].head())

    # Display the counts for each sentiment category
    print("Jumlah komentar berdasarkan sentimen:")
    print(sentiment_counts)


else:
    print("DataFrame is empty after loading. Cannot perform sentiment analysis.")

Error: File not found at /content/Book1.csv
DataFrame is empty after loading. Cannot perform sentiment analysis.


In [5]:
# Select the relevant columns for the output
sentiment_results_df = df[['Cleaned_Komentar', 'Sentiment']]

# Save the DataFrame to a CSV file
sentiment_results_df.to_csv('sentiment_results.csv', index=False)

print("Hasil analisis sentimen telah disimpan ke dalam file 'sentiment_results.csv'. Anda bisa mengunduhnya dari panel file di sebelah kiri.")

KeyError: "None of [Index(['Cleaned_Komentar', 'Sentiment'], dtype='object')] are in the [columns]"

In [ ]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import re # Assuming re is needed for cleaning if not done upstream
import nltk # Assuming nltk is needed for tokenization if not done upstream
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory # Assuming Sastrawi is needed for stop words and stemming
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory # Assuming Sastrawi is needed for stemming
from nltk.tokenize import word_tokenize # Import word_tokenize explicitly

# Re-load the original data and reapply preprocessing steps up to where df is ready

# Langkah 2: Load file CSV (replicated from cell O3y5Z5M0DXj3)
def extract_comments_from_csv(file_path):
    try:
        df = pd.read_csv(file_path, encoding='latin-1') # Added encoding
        print(f"Successfully loaded data from {file_path}") # Add success message
        return df
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return pd.DataFrame() # Return an empty DataFrame if file not found
    except UnicodeDecodeError:
        print(f"Error: UnicodeDecodeError with latin-1. Trying another encoding.")
        try:
            df = pd.read_csv(file_path, encoding='ISO-8859-1') # Try another encoding
            print(f"Successfully loaded data from {file_path} with ISO-8859-1 encoding") # Add success message
            return df
        except UnicodeDecodeError:
            print(f"Error: UnicodeDecodeError with ISO-8859-1. Trying another encoding.")
            try:
                df = pd.read_csv(file_path, encoding='cp1252') # Try another encoding
                print(f"Successfully loaded data from {file_path} with cp1252 encoding") # Add success message
                return df
            except Exception as e:
                print(f"Error reading file with cp1252 encoding: {e}")
                return pd.DataFrame()
        except Exception as e:
            print(f"Error reading file with ISO-8859-1 encoding: {e}")
            return pd.DataFrame()


# Langkah 3: Jalankan fungsi ekstraksi
file_path = '/content/Book1.csv' # Ensure this is the correct path to your CSV file
df = extract_comments_from_csv(file_path)

# Langkah 4: Tampilkan sebagian data jika DataFrame is not empty
if not df.empty:
    # Rename columns based on the first row (replicated from cell eaad2c82)
    new_columns = df.iloc[0].tolist()
    df.columns = new_columns
    df = df.drop(0).reset_index(drop=True)

    # Handle missing values and irrelevant rows (replicated from cells 61ef7bcd and c52fe95d)
    df.dropna(subset=['Isi Komentar'], inplace=True)
    other_cols = [col for col in df.columns if col != 'Isi Komentar']
    rows_to_drop = df[df[other_cols].isnull().all(axis=1)].index
    df.drop(rows_to_drop, inplace=True)

    # Text cleaning (replicated from cell 4ff00012)
    def clean_text(text):
        if pd.isna(text):
            return ""
        text = text.lower()
        text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
        text = re.sub(r'\d+', '', text)
        text = re.sub(r'[^\w\s]', '', text)
        text = text.strip()
        text = re.sub(r'\s+', ' ', text)
        return text

    df['Cleaned_Komentar'] = df['Isi Komentar'].apply(clean_text)

    # Tokenization (replicated from cell TevlVskiF2k1)
    # Ensure punkt is downloaded if needed for word_tokenize in this environment
    try:
        nltk.data.find('tokenizers/punkt')
    except LookupError:
        nltk.download('punkt')

    def tokenize_text(text):
        if not text:
            return []
        return word_tokenize(text)

    df['Tokenized_Komentar'] = df['Cleaned_Komentar'].apply(tokenize_text)

    # Stop word removal (replicated from cell bxAhPpBrF7VZ)
    stop_word_factory = StopWordRemoverFactory()
    stop_word_remover = stop_word_factory.create_stop_word_remover()

    def remove_stopwords(tokens):
        if not tokens:
            return []
        text = " ".join(tokens)
        filtered_text = stop_word_remover.remove(text)
        return filtered_text.split()

    df['Filtered_Komentar'] = df['Tokenized_Komentar'].apply(remove_stopwords)

    # Stemming (replicated from cell 6093c74f)
    stemmer_factory = StemmerFactory()
    stemmer = stemmer_factory.create_stemmer()

    def stem_tokens(tokens):
        if not tokens:
            return []
        text = " ".join(tokens)
        stemmed_text = stemmer.stem(text)
        return stemmed_text.split()

    df['Stemmed_Komentar'] = df['Filtered_Komentar'].apply(stem_tokens)


    # Calculate the value counts of the 'Platfrom' column
    if 'Platfrom' in df.columns:
        platform_counts = df['Platfrom'].value_counts()

        # Create a bar plot for platform distribution
        plt.figure(figsize=(10, 6))
        platform_counts.plot(kind='bar', color='skyblue')
        plt.title('Distribution of Platforms')
        plt.xlabel('Platform')
        plt.ylabel('Number of Comments')
        plt.xticks(rotation=45, ha='right') # Rotate labels for better readability
        plt.tight_layout() # Adjust layout
        plt.show()
    else:
        print("Column 'Platfrom' not found in the DataFrame.")

else:
    print("DataFrame is empty. Cannot plot platform distribution.")

In [ ]:
import matplotlib.pyplot as plt

# Calculate the value counts of the 'Platfrom' column
if not df.empty and 'Platfrom' in df.columns:
    platform_counts = df['Platfrom'].value_counts()

    # Create a bar plot for platform distribution
    plt.figure(figsize=(10, 6))
    platform_counts.plot(kind='bar', color='skyblue')
    plt.title('Distribution of Platforms')
    plt.xlabel('Platform')
    plt.ylabel('Number of Comments')
    plt.xticks(rotation=45, ha='right') # Rotate labels for better readability
    plt.tight_layout() # Adjust layout
    plt.show()
else:
    print("DataFrame is empty or 'Platfrom' column is missing. Cannot plot platform distribution.")

In [ ]:
import nltk
import pandas as pd
import matplotlib.pyplot as plt
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re # Import re for text cleaning
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory # Import Sastrawi for stop word removal
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory # Import Sastrawi for stemming

# Download the VADER lexicon
nltk.download('vader_lexicon')
# Download punkt for tokenization if not already downloaded
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')


# Langkah 2: Load file CSV (replicated from cell O3y5Z5M0DXj3)
def extract_comments_from_csv(file_path):
    try:
        df = pd.read_csv(file_path, encoding='latin-1') # Added encoding
        print(f"Successfully loaded data from {file_path}") # Add success message
        return df
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return pd.DataFrame() # Return an empty DataFrame if file not found
    except UnicodeDecodeError:
        print(f"Error: UnicodeDecodeError with latin-1. Trying another encoding.")
        try:
            df = pd.read_csv(file_path, encoding='ISO-8859-1') # Try another encoding
            print(f"Successfully loaded data from {file_path} with ISO-8859-1 encoding") # Add success message
            return df
        except UnicodeDecodeError:
            print(f"Error: UnicodeDecodeError with ISO-8859-1. Trying another encoding.")
            try:
                df = pd.read_csv(file_path, encoding='cp1252') # Try another encoding
                print(f"Successfully loaded data from {file_path} with cp1252 encoding") # Add success message
                return df
            except Exception as e:
                print(f"Error reading file with cp1252 encoding: {e}")
                return pd.DataFrame()
        except Exception as e:
            print(f"Error reading file with ISO-8859-1 encoding: {e}")
            return pd.DataFrame()

# Langkah 3: Jalankan fungsi ekstraksi
file_path = '/content/Book1.csv' # Ensure this is the correct path to your CSV file
df = extract_comments_from_csv(file_path)

# Langkah 4: Lanjutkan pra-pemrosesan jika DataFrame tidak kosong
if not df.empty:
    # Rename columns based on the first row (replicated from cell eaad2c82)
    new_columns = df.iloc[0].tolist()
    df.columns = new_columns
    df = df.drop(0).reset_index(drop=True)

    # Handle missing values and irrelevant rows (replicated from cells 61ef7bcd and c52fe95d)
    df.dropna(subset=['Isi Komentar'], inplace=True)
    other_cols = [col for col in df.columns if col != 'Isi Komentar']
    rows_to_drop = df[df[other_cols].isnull().all(axis=1)].index
    df.drop(rows_to_drop, inplace=True)

    # Text cleaning (replicated from cell 4ff00012)
    def clean_text(text):
        if pd.isna(text):
            return ""
        text = text.lower()
        text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
        text = re.sub(r'\d+', '', text)
        text = re.sub(r'[^\w\s]', '', text)
        text = text.strip()
        text = re.sub(r'\s+', ' ', text)
        return text

    df['Cleaned_Komentar'] = df['Isi Komentar'].apply(clean_text)

    # Tokenization (replicated from cell TevlVskiF2k1)
    def tokenize_text(text):
        if not text:
            return []
        return nltk.word_tokenize(text)

    df['Tokenized_Komentar'] = df['Cleaned_Komentar'].apply(tokenize_text)

    # Stop word removal (replicated from cell bxAhPpBrF7VZ)
    stop_word_factory = StopWordRemoverFactory()
    stop_word_remover = stop_word_factory.create_stop_word_remover()

    def remove_stopwords(tokens):
        if not tokens:
            return []
        text = " ".join(tokens)
        filtered_text = stop_word_remover.remove(text)
        return filtered_text.split()

    df['Filtered_Komentar'] = df['Tokenized_Komentar'].apply(remove_stopwords)

    # Stemming (replicated from cell 6093c74f)
    stemmer_factory = StemmerFactory()
    stemmer = stemmer_factory.create_stemmer()

    def stem_tokens(tokens):
        if not tokens:
            return []
        text = " ".join(tokens)
        stemmed_text = stemmer.stem(text)
        return stemmed_text.split()

    df['Stemmed_Komentar'] = df['Filtered_Komentar'].apply(stem_tokens)

    # Instantiate the VADER sentiment intensity analyzer
    analyzer = SentimentIntensityAnalyzer()

    # Define a function to classify sentiment
    def classify_sentiment(text):
        if pd.isna(text) or not isinstance(text, str):
            return 'Neutral'  # Classify empty or non-string text as neutral
        scores = analyzer.polarity_scores(text)
        compound_score = scores['compound']
        if compound_score >= 0.05:
            return 'Positive'
        elif compound_score <= -0.05:
            return 'Negative'
        else:
            return 'Neutral'

    # Apply the sentiment classification function to the 'Cleaned_Komentar' column
    df['Sentiment'] = df['Cleaned_Komentar'].apply(classify_sentiment)

    # 1 & 2: Calculate and plot Sentiment Distribution
    sentiment_counts = df['Sentiment'].value_counts()
    plt.figure(figsize=(8, 6))
    sentiment_counts.plot(kind='bar', color=['green', 'red', 'blue'])
    plt.title('Sentiment Distribution of Comments')
    plt.xlabel('Sentiment')
    plt.ylabel('Number of Comments')
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.show()

    # 3 & 4: Calculate and plot Platform Distribution
    if 'Platfrom' in df.columns:
        platform_counts = df['Platfrom'].value_counts()
        plt.figure(figsize=(10, 6))
        platform_counts.plot(kind='bar', color='skyblue')
        plt.title('Distribution of Platforms')
        plt.xlabel('Platform')
        plt.ylabel('Number of Comments')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()
    else:
        print("Column 'Platfrom' not found in the DataFrame. Cannot plot platform distribution.")


    # 5, 6 & 7: Flatten stemmed words and find most common words
    all_stemmed_words = [word for tokens in df['Stemmed_Komentar'] for word in tokens]
    fdist = nltk.FreqDist(all_stemmed_words)
    most_common_words = fdist.most_common(20)

    # Print the most common words
    print("Most common words:")
    print(most_common_words)

    # 8: Print numerical summary of sentiment counts
    print("\nJumlah komentar berdasarkan sentimen:")
    print(sentiment_counts)

else:
    print("DataFrame is empty after loading. Cannot perform analysis.")

In [ ]:
# Sentiment Analysis Dashboard

## Sentiment Distribution

# Display the sentiment distribution plot
# The code to generate this plot was in a previous cell:
# import matplotlib.pyplot as plt
# sentiment_counts = df['Sentiment'].value_counts()
# plt.figure(figsize=(8, 6))
# sentiment_counts.plot(kind='bar', color=['green', 'red', 'blue'])
# plt.title('Sentiment Distribution of Comments')
# plt.xlabel('Sentiment')
# plt.ylabel('Number of Comments')
# plt.xticks(rotation=0)
# plt.tight_layout()
# plt.show()

# Display the numerical sentiment counts
print("Jumlah komentar berdasarkan sentimen:")
if not df.empty and 'Sentiment' in df.columns:
    print(df['Sentiment'].value_counts())
else:
    print("DataFrame is empty or 'Sentiment' column is missing. Cannot display sentiment counts.")

**Reasoning**:
Add a Markdown heading for the platform distribution and include the code to generate the plot.



In [ ]:
## Platform Distribution

# Display the platform distribution plot
# The code to generate this plot was in a previous cell:
# import matplotlib.pyplot as plt
# platform_counts = df['Platfrom'].value_counts()
# plt.figure(figsize=(10, 6))
# platform_counts.plot(kind='bar', color='skyblue')
# plt.title('Distribution of Platforms')
# plt.xlabel('Platform')
# plt.ylabel('Number of Comments')
# plt.xticks(rotation=45, ha='right')
# plt.tight_layout()
# plt.show()

# Display the numerical platform counts
print("Jumlah komentar berdasarkan Platform:")
if not df.empty and 'Platfrom' in df.columns:
    print(df['Platfrom'].value_counts())
else:
    print("DataFrame is empty or 'Platfrom' column is missing. Cannot display platform counts.")

**Reasoning**:
Add a Markdown heading for the most common topics and include the code to print the most common words.



In [ ]:
import matplotlib.pyplot as plt

# Sentiment Distribution Plot
plt.figure(figsize=(9, 7)) # Increased figure size
sentiment_counts.plot(kind='bar', color=['#4CAF50', '#F44336', '#2196F3']) # Changed colors
plt.title('Distribusi Sentimen Komentar', fontsize=14) # Added Indonesian title, increased font size
plt.xlabel('Sentimen', fontsize=12) # Added Indonesian label, increased font size
plt.ylabel('Jumlah Komentar', fontsize=12) # Added Indonesian label, increased font size
plt.xticks(rotation=0) # Keep x-axis labels horizontal
plt.yticks(fontsize=10) # Increased y-tick font size
plt.grid(axis='y', linestyle='--', alpha=0.7) # Added a light grid for readability
plt.tight_layout()
plt.show()

# Platform Distribution Plot
plt.figure(figsize=(11, 7)) # Increased figure size
platform_counts.plot(kind='bar', color='#FFC107') # Changed color to yellow
plt.title('Distribusi Komentar Berdasarkan Platform', fontsize=14) # Added Indonesian title, increased font size
plt.xlabel('Platform', fontsize=12) # Added Indonesian label, increased font size
plt.ylabel('Jumlah Komentar', fontsize=12) # Added Indonesian label, increased font size
plt.xticks(rotation=45, ha='right', fontsize=10) # Rotate labels, align right, increased font size
plt.yticks(fontsize=10) # Increased y-tick font size
plt.grid(axis='y', linestyle='--', alpha=0.7) # Added a light grid for readability
plt.tight_layout()
plt.show()

In [ ]:
# Re-run the necessary steps to load and preprocess the data, including tokenization

import pandas as pd
import re
import nltk
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory # Keep this import as it might be needed later

# Langkah 2: Load file CSV (replicated from cell O3y5Z5M0DXj3)
def extract_comments_from_csv(file_path):
    try:
        df = pd.read_csv(file_path, encoding='latin-1') # Added encoding
        return df
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return pd.DataFrame() # Return an empty DataFrame if file not found
    except UnicodeDecodeError:
        print(f"Error: UnicodeDecodeError with latin-1. Trying another encoding.")
        try:
            df = pd.read_csv(file_path, encoding='ISO-8859-1') # Try another encoding
            return df
        except UnicodeDecodeError:
            print(f"Error: UnicodeDecodeError with ISO-8859-1. Trying another encoding.")
            try:
                df = pd.read_csv(file_path, encoding='cp1252') # Try another encoding
                return df
            except Exception as e:
                print(f"Error reading file with cp1252 encoding: {e}")
                return pd.DataFrame()
        except Exception as e:
            print(f"Error reading file with ISO-8859-1 encoding: {e}")
            return pd.DataFrame()

# Langkah 3: Jalankan fungsi ekstraksi
file_path = '/content/Book1.csv' # Ensure this is the correct path to your CSV file
df = extract_comments_from_csv(file_path)

# Langkah 4: Lanjutkan pra-pemrosesan jika DataFrame tidak kosong
if not df.empty:
    # Rename columns based on the first row (replicated from cell eaad2c82)
    new_columns = df.iloc[0].tolist()
    df.columns = new_columns
    df = df.drop(0).reset_index(drop=True)

    # Handle missing values and irrelevant rows (replicated from cells 61ef7bcd and c52fe95d)
    df.dropna(subset=['Isi Komentar'], inplace=True)
    other_cols = [col for col in df.columns if col != 'Isi Komentar']
    rows_to_drop = df[df[other_cols].isnull().all(axis=1)].index
    df.drop(rows_to_drop, inplace=True)

    # Text cleaning (replicated from cell 4ff00012)
    def clean_text(text):
        if pd.isna(text):
            return ""
        text = text.lower()
        text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
        text = re.sub(r'\d+', '', text)
        text = re.sub(r'[^\w\s]', '', text)
        text = text.strip()
        text = re.sub(r'\s+', ' ', text)
        return text

    df['Cleaned_Komentar'] = df['Isi Komentar'].apply(clean_text)

    # Tokenization (replicated from cell TevlVskiF2k1)
    try:
        nltk.data.find('tokenizers/punkt')
    except LookupError:
        nltk.download('punkt')

    def tokenize_text(text):
        if not text:
            return []
        return nltk.word_tokenize(text)

    df['Tokenized_Komentar'] = df['Cleaned_Komentar'].apply(tokenize_text)

    # Display the original and tokenized comments
    display(df[['Isi Komentar', 'Tokenized_Komentar']].head())

else:
    print("DataFrame is empty after loading. Cannot display tokenized comments.")

In [ ]:
# Execute the code in cell cb2f7226 to display the platform distribution plot
# This cell contains the code to load data, preprocess, and plot platform distribution.
# I will replicate the plotting part here for clarity and execution order.

import matplotlib.pyplot as plt

# Calculate the value counts of the 'Platfrom' column
if not df.empty and 'Platfrom' in df.columns:
    platform_counts = df['Platfrom'].value_counts()

    # Create a bar plot for platform distribution
    plt.figure(figsize=(10, 6))
    platform_counts.plot(kind='bar', color='skyblue')
    plt.title('Distribution of Platforms')
    plt.xlabel('Platform')
    plt.ylabel('Number of Comments')
    plt.xticks(rotation=45, ha='right') # Rotate labels for better readability
    plt.tight_layout() # Adjust layout
    plt.show()
else:
    print("DataFrame is empty or 'Platfrom' column is missing. Cannot plot platform distribution.")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.countplot(data=df, x='Sentiment', palette='viridis')
plt.title('Distribusi Sentimen Komentar')
plt.xlabel('Kategori Sentimen')
plt.ylabel('Jumlah Komentar')
plt.tight_layout()
plt.show()